### Введение:
В этом проекте нам необходимо провнализировать имеющуюся базу данных. Эта информация нам нужна, для того, чтобы предложить новый качественный продукт. 

In [ ]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

Импортировали библиотеки, далее следует посмотреть на данные.

In [ ]:
query = '''
             SELECT * FROM authors
        '''
authors = pd.io.sql.read_sql(sql=text(query), con = con)

In [ ]:
authors

In [ ]:
query = '''
             SELECT * FROM publishers
        '''
publishers = pd.io.sql.read_sql(sql=text(query), con = con)

In [ ]:
publishers

In [ ]:
query = '''
             SELECT * FROM ratings
        '''
ratings = pd.io.sql.read_sql(sql=text(query), con = con)

In [ ]:
ratings

In [ ]:
query = '''
             SELECT * FROM reviews
        '''
reviews = pd.io.sql.read_sql(sql=text(query), con = con)

In [ ]:
reviews

### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [ ]:
books_c = '''
SELECT COUNT(title) AS books_c FROM books
WHERE publication_date  > '2000-01-01'
'''
pd.io.sql.read_sql(books_c,con = engine)

Выполнили запрос, видим, что количество книг вышедших после 2000-01-01 составляет 819.

### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [ ]:
cnt_avg_books = '''
SELECT
     books.title AS title,
     books.book_id AS id,
     AVG(ratings.rating) AS avg_rating,
     COUNT(DISTINCT reviews.text) AS count_reviews
                        
FROM  
    books
     LEFT JOIN ratings ON ratings.book_id = books.book_id
     LEFT JOIN reviews ON reviews.book_id = books.book_id
                    
GROUP BY 
     books.title,
     books.book_id
 ORDER BY 
      count_reviews DESC
                     '''
cnt_avg_books = pd.io.sql.read_sql(cnt_avg_books, con = engine)
cnt_avg_books

Посчитали обзоры и среднюю оценку, получили по количеству ревью лидирует Сумерки(twilight), далее идет water for elephants(6), следом The glass castle. всего получилось 1000

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [ ]:
publ_50 = '''
SELECT
    publishers.publisher_id,
    publishers.publisher,
    COUNT(books.book_id) as count_books
FROM 
    publishers
JOIN books ON publishers.publisher_id = books.publisher_id
WHERE
books.num_pages > 50
GROUP BY publishers.publisher_id
ORDER BY count_books DESC
LIMIT 5;
'''
publ_50 = pd.io.sql.read_sql(publ_50, con = engine)
publ_50

Больше всего книг 50+ страниц выпустило издательство Penguin books(42), следом Vintage

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [ ]:
rate_author = '''
SELECT
    authors.author_id AS id,
    authors.author AS author,
    AVG(ratings.rating) AS avg_rate,
    COUNT(ratings.rating) AS rate
FROM
authors
    INNER JOIN books ON books.author_id = authors.author_id
    INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
id,
author
HAVING 
count(ratings.rating) >= 50
ORDER BY rate desc
limit 1;
'''
rate_author = pd.io.sql.read_sql(rate_author, con = engine)
rate_author

J.K. Rowling имеет средний рейтинг 4.288 

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [ ]:
avg_rate_reviews = '''
SELECT ROUND(avg(count))
FROM
(SELECT
    COUNT(text)
FROM reviews
    FULL OUTER JOIN(SELECT username,
        count(rating_id) as countt
FROM 
    ratings
GROUP BY
        username) AS cnt_ratings
ON cnt_ratings.username = reviews.username
where countt > 48
GROUP BY reviews.username) as count;
'''
avg_rate_reviews = pd.io.sql.read_sql(avg_rate_reviews, con = engine)
avg_rate_reviews

Cреднее кол-во обзоров от пользователей, которые поставили более 48 оценок равно 24.

### Выводы
Количество книг вышедших после 2000-01-01 составляет 819.
Посчитали обзоры и среднюю оценку, получили по количеству ревью лидирует Сумерки(twilight), далее идет water for elephants(6), следом The glass castle. всего получилось 1000.
J.K. Rowling имеет средний рейтинг 4.288
Больше всего книг 50+ страниц выпустило издательство Penguin books(42), следом Vintage
Cреднее кол-во обзоров от пользователей, которые поставили более 48 оценок равно 24.